<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Event-Logs" data-toc-modified-id="Event-Logs-2">Event Logs</a></span></li><li><span><a href="#Trace-Splitting" data-toc-modified-id="Trace-Splitting-3">Trace Splitting</a></span></li><li><span><a href="#Encoding-Techniques" data-toc-modified-id="Encoding-Techniques-4">Encoding Techniques</a></span><ul class="toc-item"><li><span><a href="#PPObj" data-toc-modified-id="PPObj-4.1">PPObj</a></span></li><li><span><a href="#Categorization" data-toc-modified-id="Categorization-4.2">Categorization</a></span></li><li><span><a href="#Fill-Missing" data-toc-modified-id="Fill-Missing-4.3">Fill Missing</a></span></li><li><span><a href="#Z-score" data-toc-modified-id="Z-score-4.4">Z-score</a></span></li><li><span><a href="#Date-conversion" data-toc-modified-id="Date-conversion-4.5">Date conversion</a></span></li><li><span><a href="#MinMax-Scaling" data-toc-modified-id="MinMax-Scaling-4.6">MinMax Scaling</a></span></li><li><span><a href="#One-HoT-Encoding" data-toc-modified-id="One-HoT-Encoding-4.7">One HoT Encoding</a></span></li></ul></li><li><span><a href="#Sub-sequence-Generation" data-toc-modified-id="Sub-sequence-Generation-5">Sub-sequence Generation</a></span></li><li><span><a href="#Data-Loader" data-toc-modified-id="Data-Loader-6">Data Loader</a></span><ul class="toc-item"><li><span><a href="#Integration-Samples" data-toc-modified-id="Integration-Samples-6.1">Integration Samples</a></span></li></ul></li></ul></div>

In [87]:
# default_exp preprocessing

Pre-processing
===
This notebook contains all relevant pre-processing. The implementation is based on the tabular notebooks in the fastai library.


In [1]:
#hide

%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

%matplotlib inline

In [2]:
#export
from mppn.imports import *

## Event Logs



All considered event logs are stored in `./event_logs`. The `EventLogs` class is a utility class to access each dataset. Logs can be loaded with the function `import_log`.

In [3]:
#export
class EventLogs:
    Helpdesk=Path('./event_logs/Helpdesk.csv')
    BPIC_12=Path('./event_logs/BPIC12.csv')
    BPIC_12_W=Path('./event_logs/BPIC12_W.csv')
    BPIC_12_Wcomplete=Path('./event_logs/BPIC12_Wc.csv')
    BPIC_12_A=Path('./event_logs/BPIC12_A.csv')
    BPIC_12_O=Path('./event_logs/BPIC12_O.csv')
    BPIC_13_CP=Path('./event_logs/BPIC13_CP.csv')
    BPIC_17_OFFER=Path('./event_logs/BPIC17_O.csv')
    BPIC_20_RFP=Path('./event_logs/BPIC20_RFP.csv')
    Mobis=Path('./event_logs/Mobis.csv')

def import_log(ds): return pd.read_csv(ds,index_col=0)

In [4]:
event_df=import_log(EventLogs.BPIC_12_W)
print(len(event_df))
event_df.head()

170107


,event_id,resource,timestamp,activity,REG_DATE,AMOUNT_REQ
trace_id,,,,,,
173688,0,112.0,2011-09-30 22:39:38.875000+00:00,W_Completeren aanvraag_SCHEDULE,2011-10-01 00:38:44.546000+02:00,20000
173688,1,NaN,2011-10-01 09:36:46.437000+00:00,W_Completeren aanvraag_START,2011-10-01 00:38:44.546000+02:00,20000
173688,2,NaN,2011-10-01 09:45:11.554000+00:00,W_Nabellen offertes_SCHEDULE,2011-10-01 00:38:44.546000+02:00,20000
173688,3,NaN,2011-10-01 09:45:13.917000+00:00,W_Completeren aanvraag_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,4,NaN,2011-10-01 10:15:41.290000+00:00,W_Nabellen offertes_START,2011-10-01 00:38:44.546000+02:00,20000


## Trace Splitting
i.e. splitting in training, validation and test set

The `split_traces` function is used to split an event_log into training, validation and test set. Furthermore, it removes traces that are longer than a specific threshhold.

In [5]:
#export
def drop_long_traces(df,max_trace_len=64,event_id='event_id'):
    df=df.drop(np.unique(df[df[event_id]>max_trace_len].index))
    return df

In [6]:
#export
def RandomTraceSplitter(split_pct=0.2, seed=None):
    "Create function that splits `items` between train/val with `valid_pct` randomly."
    def _inner(trace_ids):
        o=np.unique(trace_ids)
        np.random.seed(seed)
        rand_idx = np.random.permutation(o)
        cut = int(split_pct * len(o))
        return L(rand_idx[cut:].tolist()),L(rand_idx[:cut].tolist())
    return _inner

In [7]:
#export
def split_traces(df,df_name='tmp',test_seed=42,validation_seed=None):
    df=drop_long_traces(df)
    ts=RandomTraceSplitter(seed=test_seed)
    train,test=ts(df.index)
    ts=RandomTraceSplitter(seed=validation_seed,split_pct=0.1)
    train,valid=ts(train)
    return train,valid,test

In [8]:
#hide
a1,b1,c1=split_traces(event_df)
a2,b2,c2=split_traces(event_df)
test_ne(a1,a2),test_ne(b1,b2),test_eq(c1,c2);

In [9]:
#hide
event_df=import_log(EventLogs.BPIC_12_Wcomplete)
event_df.head()

,event_id,resource,timestamp,activity,REG_DATE,AMOUNT_REQ
trace_id,,,,,,
173688,0,NaN,2011-10-01 09:45:13.917000+00:00,W_Completeren aanvraag_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,1,NaN,2011-10-01 10:17:08.924000+00:00,W_Nabellen offertes_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,2,10913.0,2011-10-08 14:32:00.886000+00:00,W_Nabellen offertes_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,3,11049.0,2011-10-10 09:33:05.791000+00:00,W_Nabellen offertes_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,4,10629.0,2011-10-13 08:37:37.026000+00:00,W_Valideren aanvraag_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000


In [10]:
#hide
x=split_traces(event_df)
(len(np.unique(event_df.index)),len(sum(x,[])))

(9658, 9651)

## Encoding Techniques
Categorization, Normalization, One-Hot, etc.


### PPObj
an object, that manages the pre-processing and knows date columns, cat columns and cont columns
with a few convenient functions

In [11]:
#export
class _TraceIloc:
    "Get/set rows by iloc and cols by name"
    def __init__(self,o): self.o = o
    def __getitem__(self, idxs):
        df = self.o.items
        if isinstance(idxs,tuple):
            rows,cols = idxs
            rows=df.index[rows]
            return self.o.new(df.loc[rows,cols])
        else:
            rows,cols = idxs,slice(None)
            rows=np.unique(df.index)[rows]
            return self.o.new(df.loc[rows])

In [12]:
#export
class PPObj(CollBase, GetAttr, FilteredBase):
    "Main Class for Process Prediction"
    _default,with_cont='procs',True
    def __init__(self,df,procs=None,cat_names=None,cont_names=None,date_names=None,y_names=None,splits=None,
                 ycat_names=None,ycont_names=None,inplace=False,do_setup=True):
        if not inplace: df=df.copy()
        if splits is not None: df = df.loc[sum(splits, [])] # Can drop traces
        self.event_ids=df['event_id'].values if hasattr(df,'event_id') else None

        super().__init__(df)

        self.cat_names,self.cont_names,self.date_names=(L(cat_names),L(cont_names),L(date_names))
        self.set_y_names(y_names,ycat_names,ycont_names)

        self.procs = Pipeline(procs)
        self.splits=splits
        if do_setup: self.setup()


    @property
    def y_names(self): return self.ycat_names+self.ycont_names

    def set_y_names(self,y_names,ycat_names=None,ycont_names=None):
        if ycat_names or ycont_names: store_attr('ycat_names,ycont_names')
        else:
            self.ycat_names,self.ycont_names=(L([i for i in L(y_names) if i in self.cat_names]),
                                                L([i for i in L(y_names) if i not in self.cat_names]))
    def setup(self): self.procs.setup(self)
    def subset(self, i): return self.new(self.loc[self.splits[i]]) if self.splits else self
    def __len__(self): return len(np.unique(self.items.index))
    def show(self, max_n=3, **kwargs):
        print('#traces:',len(self),'#events:',len(self.items))
        display_df(self.new(self.all_cols[:max_n]).items)
    def new(self, df):
        return type(self)(df, do_setup=False,
                          **attrdict(self, 'procs','cat_names','cont_names','ycat_names','ycont_names',
                                     'date_names'))
    def process(self): self.procs(self)
    def loc(self): return self.items.loc
    def iloc(self): return _TraceIloc(self)
    def x_names (self): return self.cat_names + self.cont_names
    def all_col_names(self): return ((self.x_names+self.y_names)).unique()
    def transform(self, cols, f, all_col=True):
        if not all_col: cols = [c for c in cols if c in self.items.columns]
        if len(cols) > 0: self[cols] = self[cols].transform(f)
    def new_empty(self): return self.new(pd.DataFrame({}, columns=self.items.columns))
    def subsets(self): return [self.subset(i) for i in range(len(self.splits))] if self.splits else L(self)
properties(PPObj,'loc','iloc','x_names','all_col_names')

def _add_prop(cls, nm):
    @property
    def f(o): return o[list(getattr(o,nm+'_names'))]
    @f.setter
    def fset(o, v): o[getattr(o,nm+'_names')] = v
    setattr(cls, nm+'s', f)
    setattr(cls, nm+'s', fset)

_add_prop(PPObj, 'cat')
_add_prop(PPObj, 'cont')
_add_prop(PPObj, 'ycat')
_add_prop(PPObj, 'ycont')
_add_prop(PPObj, 'y')
_add_prop(PPObj, 'x')
_add_prop(PPObj, 'all_col')

In [13]:
ppObj=PPObj(event_df,cat_names=['activity', 'resource'],y_names=['activity'])

In [14]:
ppObj.ycat_names

(#1) ['activity']

In [15]:
ppObj.iloc[0].show() # shows first trace

#traces: 1 #events: 5


,activity,resource
trace_id,,
173688,W_Completeren aanvraag_COMPLETE,NaN
173688,W_Nabellen offertes_COMPLETE,NaN
173688,W_Nabellen offertes_COMPLETE,10913.0


We can define various pre-processing functions that are executed, when `PPOBj` is instantiated. `PPProc` is the base class for a pre-processing function. It ensures, that setup of a pre-processing function is performed using the training set, and than it is applied to the validation and test set, with the same parameters.

In [16]:
#export
class PPProc(InplaceTransform):
    "Base class to write a non-lazy tabular processor for dataframes"
    def setup(self, items=None, train_setup=False): #TODO: properly deal with train_setup
        super().setup(getattr(items,'train',items), train_setup=False)
        #super().setup(items, train_setup=False)

        # Procs are called as soon as data is available
        return self(items.items if isinstance(items,Datasets) else items)

    @property
    def name(self): return f"{super().name} -- {getattr(self,'__stored_args__',{})}"

### Categorization
i.e ordinal encoding

Implementation of ordinal or integer encoding. Adds NA values for unknown data. Implementation is pretty much taken from fastai.

In [17]:
#export
def _apply_cats (voc, add, c):
    if not is_categorical_dtype(c):
        return pd.Categorical(c, categories=voc[c.name][add:]).codes+add
    return c.cat.codes+add #if is_categorical_dtype(c) else c.map(voc[c.name].o2i)

In [18]:
#export
class Categorify(PPProc):
    "Transform the categorical variables to something similar to `pd.Categorical`"
    order = 2
    def setups(self, to):
        store_attr(classes={n:CategoryMap(to.items.loc[:,n], add_na=True) for n in to.cat_names}, but='to')
    def encodes(self, to):
        to.transform(to.cat_names, partial(_apply_cats, self.classes, 1))
    def __getitem__(self,k): return self.classes[k]

In [19]:
log=import_log(EventLogs.BPIC_12)
traces=split_traces(log)[0][:100]
splits=traces[:60],traces[60:80],traces[80:100]
o=PPObj(log,None,cat_names='activity',splits=splits)

In [20]:
m=CategoryMap(o.items.loc[:,'activity'])
len(m)

32

In [21]:
cat=Categorify()
cat.setup(o)
len(cat['activity'])

33

In [22]:
df = pd.DataFrame({'a':[0,1,2,0,2]})
to = PPObj(df, Categorify, 'a')
to.show()

#traces: 5 #events: 5


,a
0,1
1,2
2,3


In [23]:
log=import_log(EventLogs.BPIC_12)
o=PPObj(log,Categorify,'activity')

### Fill Missing
for continuous values

A pre-processing function that deals with missing data in continuous attributes. Missing data can be replaced with the median, mean or a constant value. Additionaly, we can create another boolean column that indicates, which rows were missing.  Implementation is pretty much taken from fastai.

In [38]:
#export
class FillStrategy:
    "Namespace containing the various filling strategies."
    def median  (c,fill): return c.median()
    def constant(c,fill): return fill
    def mode    (c,fill): return c.dropna().value_counts().idxmax()

In [39]:
#export
class FillMissing(PPProc):
    order=1
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_vals=None):
        if fill_vals is None: fill_vals = defaultdict(int)
        store_attr()

    def setups(self, dsets):
        missing = pd.isnull(dsets.conts).any()
        store_attr(but='to', na_dict={n:self.fill_strategy(dsets[n], self.fill_vals[n])
                            for n in missing[missing].keys()})
        self.fill_strategy = self.fill_strategy.__name__

    def encodes(self, to):
        missing = pd.isnull(to.conts)
        for n in missing.any()[missing.any()].keys():
            assert n in self.na_dict, f"nan values in `{n}` but not in setup training set"
        for n in self.na_dict.keys():
            to[n].fillna(self.na_dict[n], inplace=True)
            if self.add_col:
                to.loc[:,n+'_na'] = missing[n]
                if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')

In [40]:
fill = FillMissing() 
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4], 'b': [0,1,2,3,4,5,6]})
to = PPObj(df, fill, cont_names=['a', 'b'])
to.show()

#traces: 7 #events: 7


,a_na,a,b
0,False,0.0,0
1,False,1.0,1
2,True,1.5,2


### Z-score

Calculates standartization, also known as z-score formula. Copied from fastai.

In [41]:
#export
class Normalize(PPProc):
    "Normalize with z-score"
    order = 3
    def setups(self, to):
        store_attr(but='to', means=dict(getattr(to, 'train', to).conts.mean()),
                   stds=dict(getattr(to, 'train', to).conts.std(ddof=0)+1e-7))
        return self(to)

    def encodes(self, to): to.conts = (to.conts-self.means) / self.stds
    def decodes(self, to): to.conts = (to.conts*self.stds ) + self.means

In [42]:
event_df.head()

,event_id,activity,type,user,cost,timestamp,resource
trace_id,,,,,,,
1,0,pay expenses_start,Accounting,FI12,167.52,2017-01-16 12:29:00+00:00,FI12
1,1,pay expenses_complete,Accounting,FI12,167.52,2017-01-16 12:40:00+00:00,FI12
5,0,pay expenses_start,Accounting,JH2172,262.11,2017-01-16 07:38:00+00:00,JH2172
5,1,pay expenses_complete,Accounting,JH2172,262.11,2017-01-16 07:48:00+00:00,JH2172
6,0,calculate payments_start,Accounting,WE5108,413.14,2017-01-04 05:59:00+00:00,WE5108


In [43]:
df = pd.DataFrame({'a':[0,1,9,3,4]})
to = PPObj(df, Normalize(), cont_names='a')
to.show()

#traces: 5 #events: 5


,a
0,-1.429409
1,-1.327783
2,-0.514775


### Date conversion

Encodes a date column. Supports multiple information by using pandas date functions. This implementation is also based on the fastai but also supports relative duration from the first event of a case.

In [24]:
#export
def _make_date(df, date_field):
    "Make sure `df[date_field]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True,utc=True)

In [25]:
df = pd.DataFrame({'fu': ['2019-12-04', '2019-11-29', '2019-11-15', '2019-10-24']})
_make_date(df, 'fu')
df.dtypes

fu    datetime64[ns, UTC]
dtype: object

In [26]:
#export
def _secSinceSunNoon(datTimStr):
    dt = pd.to_datetime(datTimStr).dt
    return (dt.dayofweek-1)*24*3600+ dt.hour * 3600 + dt.minute * 60 + dt.second

In [27]:
#export
def _secSinceNoon(datTimStr):
    dt = pd.to_datetime(datTimStr).dt
    return dt.hour * 3600 + dt.minute * 60 + dt.second

In [28]:
#export
Base_Date_Encodings=['Year', 'Month', 'Day', 'Dayofweek', 'Dayofyear','Elapsed']

In [29]:
#export
def encode_date(df, field_name,unit=1e9,date_encodings=Base_Date_Encodings):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    _make_date(df, field_name)
    field = df[field_name]
    prefix =  re.sub('[Dd]ate$', '', field_name+"_")
    attr = ['Year', 'Month', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr:
        if n in date_encodings: df[prefix + n] = getattr(field.dt, n.lower())
    # Pandas removed `dt.week` in v1.1.10

    if 'secSinceSunNoon' in date_encodings:
        df[prefix+'secSinceSunNoon']=_secSinceSunNoon(field)
    if 'secSinceNoon' in date_encodings:
        df[prefix+'secSinceNoon']=_secSinceNoon(field)
    if 'Week' in date_encodings:
        week = field.dt.isocalendar().week if hasattr(field.dt, 'isocalendar') else field.dt.week
        df.insert(3, prefix+'Week', week)
    mask = ~field.isna()
    elapsed = pd.Series(np.where(mask,field.values.astype(np.int64) // unit,None).astype(float),index=field.index)

    if 'Relative_elapsed' in date_encodings:
        df[prefix+'Relative_elapsed']=elapsed-elapsed.groupby(elapsed.index).transform('min')

    # required to decode!
    if 'Elapsed' in date_encodings: df[prefix+'Elapsed']=elapsed

    df.drop(field_name, axis=1, inplace=True)
    return [],[prefix+i for i in date_encodings]

In [30]:
df = pd.DataFrame({'fu': ['2019-12-04', '2019-11-29', '2019-11-15', '2019-10-24']})
encode_date(df,'fu')
df

,fu_Year,fu_Month,fu_Day,fu_Dayofweek,fu_Dayofyear,fu_Elapsed
0,2019,12,4,2,338,1.575418e+09
1,2019,11,29,4,333,1.574986e+09
2,2019,11,15,4,319,1.573776e+09
3,2019,10,24,3,297,1.571875e+09


In [31]:
#export
def decode_date(df, field_name,unit=1e9,date_encodings=Base_Date_Encodings):
    df[field_name]=(df[field_name+'_'+'Elapsed'] * unit).astype('datetime64[ns, UTC]')
    for c in date_encodings: del df[field_name+'_'+c]

In [32]:
decode_date(df,'fu')
df

,fu
0,2019-12-04 00:00:00+00:00
1,2019-11-29 00:00:00+00:00
2,2019-11-15 00:00:00+00:00
3,2019-10-24 00:00:00+00:00


In [33]:
#export
class Datetify(PPProc):
    "Encode dates, "
    order = 0

    def __init__(self, date_encodings=['Relative_elapsed']): self.date_encodings=listify(date_encodings)

    def encodes(self, o):
        for i in o.date_names:
            cat,cont=encode_date(o.items,i,date_encodings=self.date_encodings)
            o.cont_names+=cont
            o.cat_names+=cat
# Todo: Add decoding

In [34]:
df = pd.DataFrame({'fu': ['2019-10-04', '2019-10-09', '2019-10-15', '2019-10-24']},index=[1,1,1,1])
o = PPObj(df,Datetify(date_encodings=['secSinceSunNoon','secSinceNoon','Relative_elapsed']),date_names='fu')
o.xs

,fu_secSinceSunNoon,fu_secSinceNoon,fu_Relative_elapsed
1,259200,0,0.0
1,86400,0,432000.0
1,0,0,950400.0
1,172800,0,1728000.0


### MinMax Scaling

Calculates the MinMax scaling from a column.

In [35]:
#export
class MinMax(PPProc):
    order=3

    def setups(self, o):
        store_attr(mins=o.xs.min(),
                   maxs=o.xs.max())

    def encodes(self, o):
        cols=[i+'_minmax' for i in o.x_names]
        o[cols] = o.xs.astype(float)
        o[cols] = ((o.xs-self.mins) /(self.maxs-self.mins))
        o.cont_names=L(cols)
        o.cat_names=L()

In [36]:
event_df=import_log(EventLogs.Mobis)

In [44]:
o=PPObj(event_df,[Categorify,MinMax,Datetify,FillMissing],cont_names=['cost'],cat_names=['activity'])

In [45]:
o.xs.max()

activity_minmax    1.0
cost_na_minmax     1.0
cost_minmax        1.0
dtype: float64

In [46]:
event_df=import_log(EventLogs.BPIC_12)

In [47]:
PPObj(event_df,[Categorify(),Datetify(),MinMax()],
      date_names=['timestamp'],cat_names=['activity','resource'],cont_names=['AMOUNT_REQ']).show(max_n=5)

#traces: 13087 #events: 262200


,activity_minmax,resource_minmax,AMOUNT_REQ_minmax,timestamp_Relative_elapsed_minmax
trace_id,,,,
173688,0.257143,0.014706,0.200002,0.000000
173688,0.171429,0.014706,0.200002,0.000000
173688,0.200000,0.014706,0.200002,0.000004
173688,0.685714,0.014706,0.200002,0.000005
173688,0.714286,0.000000,0.200002,0.003330


### One HoT Encoding

Calculates the one-hot encoding of a column. It is required to first apply categorization on the same column, to deal with missing values.

In [48]:
#export
from sklearn.preprocessing import OneHotEncoder

In [49]:
o=PPObj(event_df,[Categorify],cat_names=['activity','resource'])

In [50]:
len(o.xs),len(o.procs.categorify['activity'])

(262200, 37)

In [51]:
o.xs.values

array([[10,  1],
       [ 7,  1],
       [ 8,  1],
       ...,
       [20, 49],
       [ 5, 49],
       [18, 49]], dtype=int8)

In [52]:
x=o.xs.to_numpy()
categories=[range(len(o.procs.categorify['activity'])),range(len(o.procs.categorify['activity']))]

In [53]:
x=np.array(['a1','a2'])
categories=[['a1','a2','a3']]

In [54]:
ohe = OneHotEncoder(categories=categories)
a=ohe.fit_transform(x.reshape(-1, 1)).toarray()
a.shape

(2, 3)

In [55]:
categories=['a1','a2','a3']

In [56]:
#export
class OneHot(PPProc):
    "Transform the categorical variables to one-hot. Requires Categorify to deal with unseen data."
    order = 3

    def encodes(self, o):
        new_cats=[]
        for c in o.cat_names:
            categories=[range(len(o.procs.categorify[c]))]
            x=o[c].to_numpy()
            ohe = OneHotEncoder(categories=categories)
            enc=ohe.fit_transform(x.reshape(-1, 1)).toarray()
            for i in range(enc.shape[1]):
                new_cat=f'{c}_{i}'
                o.items.loc[:,new_cat]=enc[:,i]
                new_cats.append(new_cat)
        o.cat_names=L(new_cats)

In [57]:
event_df=import_log(EventLogs.BPIC_17_OFFER)

In [58]:
%%time
o=PPObj(event_df,[Categorify(),OneHot()],cat_names=['activity','resource'])

CPU times: user 407 ms, sys: 136 ms, total: 543 ms
Wall time: 543 ms


## Sub-sequence Generation

Here, the log dataframe is converted into subsequences or prefices of the cases. For a case with n events, we create n prefixes. The prefix generation is done in an efficient way, through the `np.roll` function, which shifts a numpy array by 1 element.

In [59]:
#export
def _shift_columns (a,ws=3): return np.dstack(list(reversed([np.roll(a,i) for i in range(0,ws)])))[0]

In [60]:
#export
def subsequences_fast(df,event_ids,ws=None,min_ws=64):
    max_trace_len=int(event_ids.max())+1

    if not ws: ws=max_trace_len-1
    elif ws <max_trace_len-1: raise ValueError(f"ws must be greater equal {max_trace_len-1}")
    pad=ws
    ws=max(min_ws,ws)
    trace_start = np.where(event_ids == 0)[0]
    trace_len=np.array([trace_start[i]-trace_start[i-1] for i in range(1,len(trace_start))]+[len(df)-trace_start[-1]])
    tmp=np.stack([_shift_columns(df[i],ws=ws) for i in list(df)])
    idx=[range(trace_start[i],trace_start[i]+trace_len[i]-1) for i in range(len(trace_start))]
    idx=np.array([y for x in idx for y in x])

    res=np.rollaxis(tmp,1)[idx]
    mask=ws-1-event_ids[idx][:,None] > np.arange(res.shape[2])
    res[np.broadcast_to(mask[:,None],res.shape)]=0
    return res,idx+1

In [61]:
event_df=import_log(EventLogs.Helpdesk)

In [62]:
o=PPObj(event_df,Categorify(),cat_names=['activity','resource'],y_names='activity')
#o=o.iloc[0]
len(o)

4580

In [63]:
len(o.items)-len(o)

16768

In [64]:
ws,idx=subsequences_fast(o.xs,o.event_ids,min_ws=14)
ws,ws.shape

(array([[[ 0,  0,  0, ...,  0,  0,  1],
         [ 0,  0,  0, ...,  0,  0,  1]],
 
        [[ 0,  0,  0, ...,  0,  1, 12],
         [ 0,  0,  0, ...,  0,  1,  1]],
 
        [[ 0,  0,  0, ...,  1, 12, 12],
         [ 0,  0,  0, ...,  1,  1, 12]],
 
        ...,
 
        [[ 0,  0,  0, ...,  0,  0, 12],
         [ 0,  0,  0, ...,  0,  0, 19]],
 
        [[ 0,  0,  0, ...,  0, 12, 14],
         [ 0,  0,  0, ...,  0, 19, 19]],
 
        [[ 0,  0,  0, ..., 12, 14, 10],
         [ 0,  0,  0, ..., 19, 19, 19]]], dtype=int8),
 (16768, 2, 14))

## Data Loader

The prefixes are converted to a `pytorch.Dataset` and than to a `DataLoader`
A batch is than represented as a tuple of the form `(x cat. attr,x cont. attr, y cat. attr., y cont attr.)`. Also, categorical attributes are converted to a long tensor and continous attributes to a float tensor.

If a dimensions of the batch is empty - e.g. the model does not use categorical input attributes - it is removed from the tuple. 

In [65]:
o=PPObj(event_df,Categorify(),cat_names=['activity','resource'],y_names='activity')
ws,idx=subsequences_fast(o.xs,o.event_ids,min_ws=14)
ws,ws.shape

(array([[[ 0,  0,  0, ...,  0,  0,  1],
         [ 0,  0,  0, ...,  0,  0,  1]],
 
        [[ 0,  0,  0, ...,  0,  1, 12],
         [ 0,  0,  0, ...,  0,  1,  1]],
 
        [[ 0,  0,  0, ...,  1, 12, 12],
         [ 0,  0,  0, ...,  1,  1, 12]],
 
        ...,
 
        [[ 0,  0,  0, ...,  0,  0, 12],
         [ 0,  0,  0, ...,  0,  0, 19]],
 
        [[ 0,  0,  0, ...,  0, 12, 14],
         [ 0,  0,  0, ...,  0, 19, 19]],
 
        [[ 0,  0,  0, ..., 12, 14, 10],
         [ 0,  0,  0, ..., 19, 19, 19]]], dtype=int8),
 (16768, 2, 14))

In [69]:
o.ys.iloc[idx].values[16765]

array([14], dtype=int8)

In [70]:
o.ys.groupby(o.items.index).transform('last').iloc[idx].values

array([[2],
       [2],
       [2],
       ...,
       [2],
       [2],
       [2]], dtype=int8)

In [71]:
outcome=False

In [72]:
if not outcome: y=o.ys.iloc[idx]
else: y=o.ys.groupby(o.items.index).transform('last').iloc[idx]
ycats=tensor(y[o.ycat_names].values).long()
yconts=tensor(y[o.ycont_names].values).float()
xcats=tensor(ws[:,len(o.cat_names):]).float()
xconts=tensor(ws[:,:len(o.cat_names)]).long()
xs=tuple([i for i in [xcats,xconts] if i.shape[1]>0])
ys=tuple([ycats[:,i] for i in range(ycats.shape[1])])+tuple([yconts[:,i] for i in range(yconts.shape[1])])
res=(*xs,ys)

In [73]:
res[-1]

(tensor([12, 12, 10,  ..., 14, 10,  2]),)

In [74]:
#export
class PPDset(torch.utils.data.Dataset):
    def __init__(self, inp):
        store_attr('inp')

    def __len__(self): return len(self.inp[0])

    def __getitem__(self, idx):
        xs=tuple([i[idx]for i in self.inp[:-1]])
        ys=tuple([i[idx]for i in self.inp[-1]])
        if len(ys)==1: ys=ys[0]
        return (*xs,ys)

In [75]:
dls=DataLoaders.from_dsets(PPDset(res))

In [76]:
xcat,y=dls.one_batch()
xcat.shape,y.shape

(torch.Size([64, 2, 14]), torch.Size([64]))

In [77]:
o=PPObj(event_df,Categorify(),cat_names=['activity','resource'],y_names='activity',splits=split_traces(event_df))

In [78]:
o.cat_names

(#2) ['activity','resource']

In [79]:
#export
@delegates(TfmdDL)
def get_dls(ppo:PPObj,windows=subsequences_fast,outcome=False,event_id='event_id',bs=64,**kwargs):
    ds=[]
    for s in ppo.subsets():
        wds,idx=windows(s.xs,s.event_ids)

        if not outcome: y=s.ys.iloc[idx]
        else: y=s.ys.groupby(s.items.index).transform('last').iloc[idx]
        ycats=tensor(y[s.ycat_names].values).long()
        yconts=tensor(y[s.ycont_names].values).float()
        xconts=tensor(wds[:,len(s.cat_names):]).float()
        xcats=tensor(wds[:,:len(s.cat_names)]).long()
        xs=tuple([i.squeeze() for i in [xcats,xconts] if i.shape[1]>0])
        ys=tuple([ycats[:,i] for i in range(ycats.shape[1])])+tuple([yconts[:,i] for i in range(yconts.shape[1])])
        ds.append(PPDset((*xs,ys)))
    return DataLoaders.from_dsets(*ds,bs=bs,**kwargs)
PPObj.get_dls= get_dls

In [80]:
dls=o.get_dls()
xb,yb=dls.one_batch()
xb.shape,yb.shape

(torch.Size([64, 2, 64]), torch.Size([64]))

### Integration Samples

This section shows, how the PPObj can be used to create a DataLoader for pedictive process analytics:

Next event prediction:  
X: 'activity'   
Y: 'activity'

In [81]:
log=import_log(EventLogs.Helpdesk)
o=PPObj(log,Categorify(),cat_names=['activity'],y_names='activity',splits=split_traces(event_df))
dls=o.get_dls(windows=partial(subsequences_fast,min_ws=0))
o.show(max_n=2)
xb,y=dls.one_batch()
xb.shape,y.shape


#traces: 4580 #events: 21348


,activity
trace_id,
Case3785,1
Case3785,12


(torch.Size([64, 14]), torch.Size([64]))

Next event prediction:  
X: 'activity','resource','duration'   
Y: 'activity'  

In [82]:
log=import_log(EventLogs.Helpdesk)
o=PPObj(log,[Categorify(),Datetify(),Normalize()],cat_names=['activity','resource'],date_names=['timestamp'],y_names='activity',splits=split_traces(event_df))
o.show(max_n=2)
dls=o.get_dls(windows=partial(subsequences_fast,min_ws=0))
xcat,xcont,y=dls.one_batch()
xcat[-1],xcont[-1],y[-1]

#traces: 4580 #events: 21348


,activity,resource,timestamp_Relative_elapsed
trace_id,,,
Case170,1,1,-0.789946
Case170,12,19,-0.751444


(tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17]]),
 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.7899]),
 tensor(12))

Next event prediction:  
X: 'activity','resource','duration'   
Y: 'activity','resource','duration'  

In [83]:
log=import_log(EventLogs.Helpdesk)
o=PPObj(log,[Categorify(),Datetify(),Normalize()],cat_names=['activity','resource'],date_names=['timestamp'],
        y_names=['activity','resource','timestamp_Relative_elapsed'],splits=split_traces(event_df))
o.show(max_n=2)
dls=o.get_dls(windows=partial(subsequences_fast,min_ws=0))
x=dls.one_batch()
xcat,xcont,y=dls.one_batch()
xcat[-1],xcont[-1],y[0][-1],y[1][-1],y[2][-1]

#traces: 4580 #events: 21348


,activity,resource,timestamp_Relative_elapsed
trace_id,,,
Case579,1,5,-0.787562
Case579,12,5,-0.787555


(tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 12,  8, 10],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 12,  5, 12]]),
 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000, -0.7876, -0.7390, -0.4320,  1.4882]),
 tensor(2),
 tensor(16),
 tensor(2.3610))

Outcome prediction:  
X:'activity','resource','duration'  
Y:'activity','resource','duration'  

In [84]:
log=import_log(EventLogs.Helpdesk)
o=PPObj(log,[Categorify(),Datetify(),Normalize()],cat_names=['activity','resource'],date_names=['timestamp'],
        y_names=['activity','resource','timestamp_Relative_elapsed'],splits=split_traces(event_df))
o.show(max_n=2)
dls=o.get_dls(windows=partial(subsequences_fast,min_ws=0),outcome=True)
xcat,xcont,y=dls.one_batch()
xcat[-1],xcont[-1],y[0][-1],y[1][-1],y[2][-1]

#traces: 4580 #events: 21348


,activity,resource,timestamp_Relative_elapsed
trace_id,,,
Case338,1,5,-0.788453
Case338,12,5,-0.731482


(tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 12, 12],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 21, 21, 22]]),
 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000, -0.7885, -0.7884, -0.7878]),
 tensor(2),
 tensor(18),
 tensor(1.9353))

Outcome prediction with One-Hot-Encoding  
X:'activity','resource'  
Y:'activity'

In [85]:
log=import_log(EventLogs.Helpdesk)
o=PPObj(log,[Categorify(),Datetify(),OneHot()],cat_names=['activity','resource'],y_names='activity',splits=split_traces(event_df))
o.show(max_n=2)
dls=o.get_dls(windows=partial(subsequences_fast,min_ws=0),outcome=True)
xcat,y=dls.one_batch()
xcat.shape,y.shape

#traces: 4580 #events: 21348


,activity_0,activity_1,activity_2,activity_3,activity_4,activity_5,activity_6,activity_7,activity_8,activity_9,activity_10,activity_11,activity_12,activity_13,activity_14,resource_0,resource_1,resource_2,resource_3,resource_4,resource_5,resource_6,resource_7,resource_8,resource_9,resource_10,resource_11,resource_12,resource_13,resource_14,resource_15,resource_16,resource_17,resource_18,resource_19,resource_20,resource_21,resource_22,activity
trace_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Case2186,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
Case2186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12


(torch.Size([64, 38, 14]), torch.Size([64]))

Outcome prediction with Min-Max-Scaling  
X:'activity','resource','duration'  
Y:'activity','resource','duration'

In [86]:
log=import_log(EventLogs.Helpdesk)
o=PPObj(log,[Categorify(),Datetify(),MinMax()],cat_names=['activity','resource'],date_names=['timestamp'],
        y_names=['activity_minmax','resource_minmax','timestamp_Relative_elapsed'],splits=split_traces(event_df))
o.show(max_n=2)
dls=o.get_dls(windows=partial(subsequences_fast,min_ws=0),outcome=True)
xcont,y=dls.one_batch()
xcat.shape,xcont.shape,y[0].shape,y[1].shape,y[2].shape

#traces: 4580 #events: 21348


,activity_minmax,resource_minmax,timestamp_Relative_elapsed_minmax,timestamp_Relative_elapsed
trace_id,,,,
Case4559,0.000000,1.0,0.000000,0.0
Case4559,0.846154,1.0,0.000023,118.0


(torch.Size([64, 38, 14]),
 torch.Size([64, 3, 14]),
 torch.Size([64]),
 torch.Size([64]),
 torch.Size([64]))